# [2] 배송지연 현황 파악
 1) 지역별 배송 지연 건수 (민석님 코드 추가 필요 -)  
 2) 지역별 배송 지연율 (민석님 코드 추가 필요-)  

## 0) 데이터 전처리

In [1]:
delay_data = raw[(raw['order_status'] == 'delivered') & (raw['is_delayed'] == 1)].groupby('customer_state')['order_id'].count().to_frame('delayed_count')
delay_per = raw[raw['order_status'] == 'delivered'].groupby('customer_state')['order_id'].count()
delay_data['pct'] = delay_data['delayed_count'] / delay_per * 100

import geopandas as gpd
import numpy as np
from adjustText import adjust_text
import matplotlib.patheffects as path_effects

brazil = gpd.read_file('br_states.json')

data = brazil.merge(delay_data, left_on='SIGLA', right_on='customer_state', how='outer')

NameError: name 'raw' is not defined

## 1) 지역별 배송 지연 건수

In [ ]:
# 코드

custom_bins = [0, 30, 60, 90, 120, 150, 200, 400, 800, 1600, 2400, 2800]
data['bin'] = pd.cut(data['is_delayed'], bins=custom_bins, labels=False)

num_bins = len(custom_bins) # data['bin'].nunique()
palette = sns.color_palette('YlOrRd', num_bins)

data['color'] = data['bin'].map(lambda x: palette[x] if pd.notnull(x) else (0.9, 0.9, 0.9))

# 지도 범위 계산 (오프셋 비율로 좌표계 단위 대응)
bounds = data.total_bounds
minx, miny, maxx, maxy = bounds
width = maxx - minx
height = maxy - miny

# 비율 오프셋 (살짝만 우상단)
x_offset_ratio = 0.01
y_offset_ratio = 0.005
x_offset = width * x_offset_ratio
y_offset = height * y_offset_ratio

# 플롯 생성
fig, ax = plt.subplots(figsize=(10, 10))

import matplotlib as mpl

norm = mpl.colors.BoundaryNorm(custom_bins, len(custom_bins)-1)
cmap = mpl.cm.get_cmap('YlOrRd', len(custom_bins)-1)

sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax, shrink=0.7)
cbar.set_label('Delay (건수)')

# 지도 그리기
data.plot(
    column='delay',
    color=data['color'],
    linewidth=0.8,
    legend=True,
    ax=ax,
    edgecolor='black',
    legend_kwds={'shrink': 0.7, 'label': '지연 건수'}
)


# 텍스트 추가
texts = []
for idx, row in data.iterrows():
    if pd.notnull(row['is_delayed']) and not row['geometry'].centroid.is_empty:
        x, y = row['geometry'].centroid.x, row['geometry'].centroid.y
        label = f"{row['SIGLA']} {int(row['is_delayed'])}건"
        t = ax.text(
            x + x_offset,
            y + y_offset,
            label,
            ha='center',
            va='center',
            fontsize=10,
            color='dimgray'
        )
        t.set_path_effects([
            path_effects.Stroke(linewidth=3, foreground='white'),  # 외곽선 더 두껍게
            path_effects.Normal(),
        ])
        texts.append(t)

# adjust_text 사용 - 화살표 없이 살짝만 밀어서 겹침 방지
adjust_text(
    texts,
    ax=ax,
    arrowprops=None,
    force_text=0.3,
    force_points=0.1
)

# 제목
ax.set_title('브라질 주 별 지연 건수')

# x, y축 라벨 제거
ax.set_xlabel('')
ax.set_ylabel('')

# x, y축 눈금 제거
ax.set_xticks([])
ax.set_yticks([])
ax.tick_params(left=False, bottom=False)

plt.show()

## 2) 지역별 배송 지연율

In [ ]:
# 코드
import geopandas as gpd

import numpy as np
from adjustText import adjust_text
import matplotlib.patheffects as path_effects


custom_bins = [0, 30, 60, 90, 120, 150, 200, 400, 800, 1600, 2400]
data['bin'] = pd.cut(data['pct'], bins=custom_bins, labels=False)

num_bins = len(custom_bins) # data['bin'].nunique()
palette = sns.color_palette('YlOrRd', num_bins)


fig, ax = plt.subplots(figsize=(10, 10))

# data.plot(column='count_deliv', cmap='OrRd', linewidth=0.8, legend=True, ax=ax, edgecolor='black')
data.plot(column='pct', cmap='Greens', linewidth=0.8, legend=True, ax=ax, edgecolor='gray', legend_kwds={'shrink': 0.7})

bounds = data.total_bounds
minx, miny, maxx, maxy = bounds
width = maxx - minx
height = maxy - miny

# 비율로 오프셋
x_offset_ratio = 0.02
y_offset_ratio = 0.01

x_offset = width * x_offset_ratio + 0.2
y_offset = height * y_offset_ratio + 0.2

# 텍스트 추가
texts = []
for idx, row in data.iterrows():
    if pd.notnull(row['pct']) and not row['geometry'].centroid.is_empty:
        x, y = row['geometry'].centroid.x, row['geometry'].centroid.y
        label = f"{row['SIGLA']} {row['pct']:.2f}%"
        t = ax.text(
            x + x_offset,
            y + y_offset,
            label,
            ha='center',
            va='center',
            fontsize=9,
            color='black'
        )
        t.set_path_effects([
            path_effects.Stroke(linewidth=1.8, foreground='white'),
            path_effects.Normal()
        ])
        texts.append(t)        # ax.text(x, y, label, ha='center', va='center', fontsize=8, color='red')

props = dict(arrowstyle=None, color='red', alpha=0.8, lw=0.8,
             shrinkA=80, shrinkB=80, connectionstyle='arc3, rad=0.1')

adjust_text(texts, ax=ax, arrowprops=None, force_text=0.5, force_points=0.2)

ax.set_title('브라질 주 별 지연율')
ax.set_xlabel = ''
ax.set_ylabel = ''
ax.set_xticks([])
ax.set_yticks([])
ax.tick_params(left=False, bottom=False)
plt.show()